In [1]:
import scanpy as sc
data_dir = '../../dataset/Marshall2022High_human_sampled.h5ad'
adata = sc.read_h5ad(data_dir)
adata.X = adata.raw.X

In [2]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [3]:
adata.var['gene_id'] = adata.var.index
adata.var.index = adata.var['gene_name']

# 输出 adata.var
adata.var

,feature_reference,feature_biotype,feature_is_filtered-0,feature_name-0,feature_length-0,feature_type-0,feature_is_filtered-1,feature_name-1,feature_length-1,feature_type-1,...,feature_is_filtered-6,feature_name-6,feature_length-6,feature_type-6,feature_is_filtered-7,feature_name-7,feature_length-7,feature_type-7,gene_name,gene_id
gene_name,,,,,,,,,,,,,,,,,,,,,
A1BG,NCBITaxon:9606,gene,True,A1BG,2134,protein_coding,False,A1BG,2134,protein_coding,...,True,A1BG,2134,protein_coding,True,A1BG,2134,protein_coding,A1BG,ENSG00000121410
A1BG-AS1,NCBITaxon:9606,gene,False,A1BG-AS1,1667,lncRNA,False,A1BG-AS1,1667,lncRNA,...,False,A1BG-AS1,1667,lncRNA,False,A1BG-AS1,1667,lncRNA,A1BG-AS1,ENSG00000268895
A1CF,NCBITaxon:9606,gene,False,A1CF,2211,protein_coding,False,A1CF,2211,protein_coding,...,False,A1CF,2211,protein_coding,False,A1CF,2211,protein_coding,A1CF,ENSG00000148584
A2M,NCBITaxon:9606,gene,False,A2M,590,protein_coding,False,A2M,590,protein_coding,...,False,A2M,590,protein_coding,False,A2M,590,protein_coding,A2M,ENSG00000175899
A2M-AS1,NCBITaxon:9606,gene,False,A2M-AS1,2551,lncRNA,False,A2M-AS1,2551,lncRNA,...,False,A2M-AS1,2551,lncRNA,False,A2M-AS1,2551,lncRNA,A2M-AS1,ENSG00000245105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZZ3,NCBITaxon:9606,gene,False,ZZZ3,686,protein_coding,False,ZZZ3,686,protein_coding,...,False,ZZZ3,686,protein_coding,False,ZZZ3,686,protein_coding,ZZZ3,ENSG00000036549
MISP3,NCBITaxon:9606,gene,False,MISP3,1473,protein_coding,False,MISP3,1473,protein_coding,...,False,MISP3,1473,protein_coding,False,MISP3,1473,protein_coding,MISP3,ENSG00000141854
ENSG00000234352.9,NCBITaxon:9606,gene,False,ENSG00000234352.9,728,lncRNA,False,ENSG00000234352.9,728,lncRNA,...,False,ENSG00000234352.9,728,lncRNA,False,ENSG00000234352.9,728,lncRNA,ENSG00000234352.9,ENSG00000234352


In [4]:
import scanpy as sc
import torch
import lightning.pytorch as pl
from self_supervision.models.lightning_modules.cellnet_autoencoder import MLPAutoEncoder
from self_supervision.estimator.cellnet import EstimatorAutoEncoder

# 设置你的 .ckpt 文件路径
ckpt_path = "../../sc_pretrained/Pretrained Models/RandomMask.ckpt"

# 模型参数
units_encoder = [512, 512, 256, 256, 64]
units_decoder = [256, 256, 512, 512]

# 初始化 EstimatorAutoEncoder 实例
estim = EstimatorAutoEncoder(data_path=None)  # 如果没有实际数据路径，可以设置为None

# 加载预训练模型
estim.model = MLPAutoEncoder.load_from_checkpoint(
    ckpt_path,
    gene_dim=19331,  # 根据你的数据调整
    batch_size=128,  # 根据你的需要调整
    units_encoder=units_encoder, 
    units_decoder=units_decoder,
    masking_strategy="random",  # 假设模型使用了随机掩码
    masking_rate=0.5,  # 根据需要调整
)

# 使用 GPU 进行评估（如果可用）
estim.trainer = pl.Trainer(accelerator="gpu", devices=1 if torch.cuda.is_available() else None)
estim.model

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs


MLPAutoEncoder(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (encoder): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=256, out_features=256, b

In [5]:
import pandas as pd
var_df = pd.read_parquet('../../sc_pretrained/var.parquet')
var_df

,feature_id,feature_name
0,ENSG00000186092,OR4F5
1,ENSG00000284733,OR4F29
2,ENSG00000284662,OR4F16
3,ENSG00000187634,SAMD11
4,ENSG00000188976,NOC2L
...,...,...
19326,ENSG00000288702,UGT1A3
19327,ENSG00000288705,UGT1A5
19328,ENSG00000182484,WASH6P
19329,ENSG00000288622,PDCD6-AHRR


In [6]:
all_genes = var_df['feature_name'].tolist()
all_genes

['OR4F5',
 'OR4F29',
 'OR4F16',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20',
 'ANKRD65',
 'TMEM88B',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'FNDC10',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'NADK',
 'GNB1',
 'CALML6',
 'TMEM52',
 'CFAP74',
 'GABRD',
 'PRKCZ',
 'FAAP20',
 'SKI',
 'MORN1',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'HES5',
 'TNFRSF14',
 'PRXL2B',
 'MMEL1',
 'TTC34',
 'ACTRT2',
 'PRDM16',
 'ARHGEF16',
 'MEGF6',
 'TPRG1L',
 'WRAP73',
 'TP73',
 'CCDC27',
 'SMIM1',
 'LRRC47',
 'CEP104',
 'DFFB',
 'C1orf174',
 'AJAP1',
 'NPHP4',
 'KCNAB2',
 'CHD5',
 'RPL22',
 'RNF207',
 'ICMT',
 'HES3',
 'GPR153',
 'ACOT7',
 'HES2',
 'ESPN',
 'TNFRSF25',
 'PLEKHG5',
 'NOL9',
 'TAS1R1',
 'ZBTB48',
 'KLH

In [7]:
adata.var['gene_name']=adata.var.index
adata.var['gene_name']

gene_name
A1BG                                              A1BG
A1BG-AS1                                      A1BG-AS1
A1CF                                              A1CF
A2M                                                A2M
A2M-AS1                                        A2M-AS1
                                       ...            
ZZZ3                                              ZZZ3
MISP3                                            MISP3
ENSG00000234352.9                    ENSG00000234352.9
ARHGAP27P1-BPTFP1-KPNA2P3    ARHGAP27P1-BPTFP1-KPNA2P3
SBNO1-AS1                                    SBNO1-AS1
Name: gene_name, Length: 20299, dtype: category
Categories (25391, object): ['7SK_ENSG00000202198', 'A1BG', 'A1BG-AS1', 'A1CF', ..., 'ZYG11B', 'ZYX', 'ZZEF1', 'ZZZ3']

In [8]:
import numpy as np
# 初始化一个新的数据矩阵，形状为 (adata.X.shape[0], len(all_genes))，填充为零
new_data = np.zeros((adata.X.shape[0], len(all_genes)), dtype=np.float32)

In [9]:
existing_genes = adata.var['gene_name']
existing_genes

gene_name
A1BG                                              A1BG
A1BG-AS1                                      A1BG-AS1
A1CF                                              A1CF
A2M                                                A2M
A2M-AS1                                        A2M-AS1
                                       ...            
ZZZ3                                              ZZZ3
MISP3                                            MISP3
ENSG00000234352.9                    ENSG00000234352.9
ARHGAP27P1-BPTFP1-KPNA2P3    ARHGAP27P1-BPTFP1-KPNA2P3
SBNO1-AS1                                    SBNO1-AS1
Name: gene_name, Length: 20299, dtype: category
Categories (25391, object): ['7SK_ENSG00000202198', 'A1BG', 'A1BG-AS1', 'A1CF', ..., 'ZYG11B', 'ZYX', 'ZZEF1', 'ZZZ3']

In [10]:
# 将所有基因名称转换为小写
all_genes_lower = [gene.lower() for gene in all_genes]
adata_genes_lower = [gene.lower() for gene in existing_genes]

# 将两个列表转换为集合
all_genes_set = set(all_genes_lower)
adata_genes_set = set(adata_genes_lower)

# 计算交集
matching_genes = all_genes_set.intersection(adata_genes_set)
matching_count = len(matching_genes)
# 计算不匹配的基因
non_matching_genes = adata_genes_set - matching_genes
non_matching_count = len(non_matching_genes)


# 输出结果
print(f"匹配的基因数量: {matching_count}")
print(f"匹配的基因列表: {matching_genes}")
non_matching_genes


匹配的基因数量: 15360
匹配的基因列表: {'zdhhc6', 'nploc4', 'swt1', 'repin1', 'znf136', 'c4orf3', 'fgfbp1', 'rnd3', 'rab3gap2', 'set', 'znf160', 'robo4', 'tasor2', 'fhl1', 'zmynd10', 'scgb1d2', 'foxm1', 'ubc', 'pgbd5', 'phc2', 'hoxb8', 'guf1', 'mif', 'gna14', 'cdk5', 'c19orf18', 'xpot', 'helz', 'rfesd', 'cdc42bpg', 'ifi27', 'gatd1', 'pex12', 'zscan5a', 'jchain', 'tuba1b', 'clstn3', 'znf83', 'phyhip', 'rad9b', 'e2f2', 'wdr37', 'cenpn', 'chrne', 'scamp5', 'slc25a23', 'mbd6', 'znf135', 'dclk2', 'nipa1', 'camsap1', 'gatad2a', 'tbkbp1', 'nsmce3', 'bcl2l14', 'jak2', 'lrrc69', 'angptl6', 'znf549', 'cnot8', 'eif2s2', 'kcng2', 'gtf2ird2b', 'pdgfrl', 'adcy1', 'frs2', 'prpf4b', 'znf233', 'ociad1', 'rapgef2', 'pdgfd', 'fabp3', 'usb1', 'nop56', 'proser1', 'atp5me', 'msmo1', 'nme6', 'asz1', 'e4f1', 'ralgapa1', 'evi2b', 'tmbim1', 'myrfl', 'chd7', 'nprl2', 'zbtb17', 'dtwd1', 'pigv', 'tspan12', 'ybx1', 'pex1', 'ddx55', 'tacc3', 'gpr155', 'xdh', 'c2cd6', 'ggct', 'pts', 'polr2i', 'rbis', 'acot12', 'wasf3', 'taf4b', 'dp

{'linc00113',
 'sdk1-as1',
 'ensg00000235127.1',
 'ensg00000229848.1',
 'skic8',
 'ensg00000267327.2',
 'alg1l9p_ensg00000248671',
 'linc01982',
 'linc02422',
 'linc01923',
 'tmem263-dt',
 'msrb3-as1',
 'ensg00000260160.1',
 'itfg2-as1_ensg00000258325',
 'ensg00000273398.6',
 'ensg00000257342.1',
 'gas6-dt',
 'cd109-as1',
 'hyi-as1',
 'ensg00000272909.1',
 'rn7skp30',
 'arhgap11b-dt',
 'ensg00000236358.1',
 'ensg00000266824.1',
 'linc02331',
 'rn7sl47p',
 'ensg00000266313.2',
 'erve-1',
 'linc01307',
 'ensg00000272918.1',
 'ensg00000273254.1',
 'ensg00000234255.9',
 'linc02596',
 'mcm3ap-as1',
 'ensg00000272558.1',
 'tesc-as1',
 'ensg00000250969.1',
 'ensg00000124593.16',
 'znf436-as1',
 'tm4sf19-as1',
 'linc00944',
 'ensg00000267940.1',
 'ensg00000271860.9',
 'pcnx4-dt',
 'linc00963',
 'ensg00000268401.2',
 'linc00957',
 'ensg00000255126.1',
 'ensg00000269807.1',
 'ensg00000272754.1',
 'baz1a-as1',
 'znf8-dt',
 'atxn7l3-as1',
 'linc02741',
 'ensg00000229151.3',
 'ensg00000248994.2',
 

In [11]:
gene_to_index = {gene: idx for idx, gene in enumerate(all_genes_lower)}
gene_to_index

{'or4f5': 0,
 'or4f29': 1,
 'or4f16': 2,
 'samd11': 3,
 'noc2l': 4,
 'klhl17': 5,
 'plekhn1': 6,
 'perm1': 7,
 'hes4': 8,
 'isg15': 9,
 'agrn': 10,
 'rnf223': 11,
 'c1orf159': 12,
 'ttll10': 13,
 'tnfrsf18': 14,
 'tnfrsf4': 15,
 'sdf4': 16,
 'b3galt6': 17,
 'c1qtnf12': 18,
 'ube2j2': 19,
 'scnn1d': 20,
 'acap3': 21,
 'pusl1': 22,
 'ints11': 23,
 'cptp': 24,
 'tas1r3': 25,
 'dvl1': 26,
 'mxra8': 27,
 'aurkaip1': 28,
 'ccnl2': 29,
 'mrpl20': 30,
 'ankrd65': 31,
 'tmem88b': 32,
 'vwa1': 33,
 'atad3c': 34,
 'atad3b': 35,
 'atad3a': 36,
 'tmem240': 37,
 'ssu72': 38,
 'fndc10': 39,
 'mib2': 40,
 'mmp23b': 41,
 'cdk11b': 42,
 'slc35e2b': 43,
 'cdk11a': 44,
 'nadk': 45,
 'gnb1': 46,
 'calml6': 47,
 'tmem52': 48,
 'cfap74': 49,
 'gabrd': 50,
 'prkcz': 51,
 'faap20': 52,
 'ski': 53,
 'morn1': 54,
 'rer1': 55,
 'pex10': 56,
 'plch2': 57,
 'pank4': 58,
 'hes5': 59,
 'tnfrsf14': 60,
 'prxl2b': 61,
 'mmel1': 62,
 'ttc34': 63,
 'actrt2': 64,
 'prdm16': 65,
 'arhgef16': 66,
 'megf6': 67,
 'tprg1l': 68

In [12]:
only_in_all_genes = all_genes_set - adata_genes_set

only_in_adata_genes = adata_genes_set - all_genes_set

# 输出结果
print(f"仅在 all_genes 中存在的基因数量: {len(only_in_all_genes)}")
print(f"仅在 all_genes 中存在的基因: {only_in_all_genes}")

print(f"仅在 adata_genes 中存在的基因数量: {len(only_in_adata_genes)}")
print(f"仅在 adata_genes 中存在的基因: {only_in_adata_genes}")


仅在 all_genes 中存在的基因数量: 3971
仅在 all_genes 中存在的基因: {'h3y1', 'bend2', 'serpinb11', 'klk4', 'npipb9', 'cdy2b', 'smyd1', 'tbx10', 'amy1c', 'prss51', 'krtap6-2', 'figla', 'psmb11', 'ppp4r3c', 'tiaf1', 'page2b', 'lgals9b', 'gpr179', 'gatd3a', 'prop1', 'lmln2', 'trarg1', 'c16orf92', 'ervw-1', 'tril', 'or9a4', 'cshl1', 'or2a5', 'mageb3', 'ccr8', 'eef1akmt4', 'mrpl45', 'ugt2b10', 'camkv', 'potee', 'or10h5', 'scrt2', 'pcsk9', 'mansc4', 'fam187b', 'hnrnpcl3', 'saa4', 'mrm1', 'gcg', 'b4galnt4', 'tmem249', 'hephl1', 'zic4', 'or8k1', 'acr', 'ugt2a1', 'stpg3', 'tmem151a', 'mtrnr2l12', 'serpinb4', 'nr0b1', 'tas2r3', 'gpr78', 'zscan5b', 'rab40al', 'cdh19', 'fcn2', 'ptx4', 'or1j1', 'nt5c1a', 'slc10a1', 'gata4', 'tas2r7', 'or52d1', 'pmf1-bglap', 'tex13a', 'mrgprx4', 'hspb2-c11orf52', 'lefty2', 'pramef4', 'ccdc173', 'smim39', 'tnfrsf13b', 'spanxn2', 'znf511-prap1', 'usp17l21', 'tbxt', 'taf11l12', 'c6orf15', 'b3galt9', 'serpina7', 'actrt2', 'snurf', 'cdrt15', 'fdcsp', 'gpr139', 'htr5a', 'prkag3', 'dspp', 'd

In [13]:
import numpy as np
from scipy.sparse import csr_matrix

# Initialize a mapping from gene names in adata to their column indices
adata_gene_to_index = {gene: idx for idx, gene in enumerate(adata_genes_lower)}

# Create an array to map adata.X column indices to new_data column indices
adata_to_new_data_indices = -1 * np.ones(adata.X.shape[1], dtype=int)
for idx, gene in enumerate(adata_genes_lower):
    if gene in gene_to_index:
        adata_to_new_data_indices[idx] = gene_to_index[gene]

In [14]:

# Extract data from adata.X without converting it to a dense array
data = adata.X.data
indices = adata.X.indices
indptr = adata.X.indptr

# Map the column indices to the new indices in new_data
mapped_indices = adata_to_new_data_indices[indices]

# Filter out entries where the mapping is invalid (-1)
valid_entries = mapped_indices != -1
new_data_values = data[valid_entries]
new_data_indices = mapped_indices[valid_entries]

# Build the new indptr array for the new_data matrix
new_indptr = np.zeros(adata.X.shape[0] + 1, dtype=int)

In [15]:
for i in range(adata.X.shape[0]):
    row_start = indptr[i]
    row_end = indptr[i + 1]
    valid_count = np.sum(valid_entries[row_start:row_end])
    new_indptr[i + 1] = new_indptr[i] + valid_count

In [16]:
# Construct the new_data sparse matrix
new_data = csr_matrix(
    (new_data_values, new_data_indices, new_indptr),
    shape=(adata.X.shape[0], len(all_genes)),
    dtype=np.float32
)


In [17]:
new_data = new_data.toarray()

In [18]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split


label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(adata.obs['cell_type'])  # 预先编码标签


random_seed = 42
X_train_val, X_test, y_train_val, y_test = train_test_split(
    new_data, labels_encoded, test_size=0.15, random_state=random_seed)


X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=random_seed)  # 0.1765 是为了让验证集占 15%

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

estim.model.eval()
with torch.no_grad():
    X_train_tensor = torch.tensor(X_train).float().to(device)
    X_test_tensor = torch.tensor(X_test).float().to(device)
    train_embeddings = estim.model.encoder(X_train_tensor).detach().cpu().numpy()
    test_embeddings = estim.model.encoder(X_test_tensor).detach().cpu().numpy()


cuda


In [19]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

    

    # 初始化和训练KNN分类器
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_embeddings, y_train)
    
    # 模型预测
predictions = knn.predict(test_embeddings)

    # 计算准确率和 F1 分数
accuracy = accuracy_score(y_test, predictions)
print(f"KNN Accuracy on Test Data: {accuracy}")
f1 = f1_score(y_test, predictions, average='weighted')
print(f"Weighted F1 Score: {f1}")
    
macro_f1 = f1_score(y_test, predictions, average='macro')
print(f'Macro F1 Score: {macro_f1}')

    # 计算随机猜测的准确率
class_probabilities = np.bincount(y_test) / len(y_test)
random_accuracy = np.sum(class_probabilities ** 2)
print(f"Random Guess Accuracy: {random_accuracy}")

    # 生成分类报告
report = classification_report(y_test, predictions, target_names=label_encoder.classes_)
print(report)

KNN Accuracy on Test Data: 0.6280804631828979
Weighted F1 Score: 0.5987555366810663
Macro F1 Score: 0.3298828508347299
Random Guess Accuracy: 0.2927884776016484
                                                           precision    recall  f1-score   support

                          blood vessel smooth muscle cell       0.32      0.17      0.22       852
                                         endothelial cell       0.65      0.84      0.74     13511
                 kidney collecting duct intercalated cell       0.40      0.28      0.33      1032
                    kidney collecting duct principal cell       0.69      0.66      0.67      2040
          kidney distal convoluted tubule epithelial cell       0.56      0.38      0.45       618
                                     kidney granular cell       0.18      0.02      0.03       105
                           kidney interstitial fibroblast       0.32      0.08      0.13       781
kidney loop of Henle thick ascending limb epit

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [20]:
with torch.no_grad():
    new_data_tensor = torch.tensor(new_data).float().to(device)
    SSL_embeddings = estim.model.encoder(new_data_tensor).detach().cpu().numpy()
new_adata = sc.read_h5ad(data_dir)
new_adata.obsm[f'SSL_RM_ZS_{random_seed}'] = SSL_embeddings
new_adata.uns[f'RM_ZS_y_test_{random_seed}'] = y_test
new_adata.uns[f'RM_ZS_predictions_{random_seed}'] = predictions
new_adata.uns[f'RM_ZS_target_names_{random_seed}'] = label_encoder.classes_
new_adata.write_h5ad(data_dir)

In [21]:

import pandas as pd
import os
import re

# 当前 Notebook 文件名
notebook_name = "slide_seq_human_kidney_random_mask_zero_shot_42.ipynb"

# 初始化需要打印的值
init_train_loss = train_losses[0] if 'train_losses' in globals() else None
init_val_loss = val_losses[0] if 'val_losses' in globals() else None
converged_epoch = len(train_losses) - patience if 'train_losses' in globals() else None
converged_val_loss = best_val_loss if 'best_val_loss' in globals() else None

# 打印所有所需的指标
print("Metrics Summary:")
if 'train_losses' in globals():
    print(f"init_train_loss\tinit_val_loss\tconverged_epoch\tconverged_val_loss\tmacro_f1\tweighted_f1\tmicor_f1")
    print(f"{init_train_loss:.3f}\t{init_val_loss:.3f}\t{converged_epoch}\t{converged_val_loss:.3f}\t{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")
else:
    print(f"macro_f1\tweighted_f1\tmicor_f1")
    print(f"{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")

# 保存结果到 CSV 文件
output_data = {
    'dataset_split_random_seed': [int(random_seed)],
    'dataset': ['slide_seq_human_kidney'],
    'method': [re.search(r'kidney_(.*?)_\d+', notebook_name).group(1)],
    'init_train_loss': [init_train_loss if init_train_loss is not None else ''],
    'init_val_loss': [init_val_loss if init_val_loss is not None else ''],
    'converged_epoch': [converged_epoch if converged_epoch is not None else ''],
    'converged_val_loss': [converged_val_loss if converged_val_loss is not None else ''],
    'macro_f1': [macro_f1],
    'weighted_f1': [f1],
    'micor_f1': [accuracy]
}
output_df = pd.DataFrame(output_data)

# 保存到当前目录下名为 results 的文件夹中
if not os.path.exists('results'):
    os.makedirs('results')

csv_filename = f"results/{os.path.splitext(notebook_name)[0]}_results.csv"
output_df.to_csv(csv_filename, index=False)


Metrics Summary:
macro_f1	weighted_f1	micor_f1
0.330	0.599	0.628
